<a href="https://colab.research.google.com/github/kingjiwoo/deep_learnig_from_scratch_1/blob/main/scaratch2_ch2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
text = 'You say goodbye and I say hello.'

In [2]:
text = text.lower() #소문자로 변경
text = text.replace('.', ' .')
text

'you say goodbye and i say hello .'

In [3]:
words = text.split(' ')
words

['you', 'say', 'goodbye', 'and', 'i', 'say', 'hello', '.']

단어에 ID를 부여하고 ID의 리스트로 이용할 수 있도록 손질

In [4]:
word_to_id = {}
id_to_word = {}

for word in words:
    if word not in word_to_id:
        new_id = len(word_to_id)
        word_to_id[word] = new_id # 키가 word 밸류가 id
        id_to_word[new_id] = word # 키가 id 밸류가 word

In [6]:
id_to_word
word_to_id

{'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hello': 5, '.': 6}

In [8]:
# corpus로 만들기 일종의 임베딩
import numpy as np
corpus = [word_to_id[w] for w in words]
corpus = np.array(corpus)
corpus

array([0, 1, 2, 3, 4, 1, 5, 6])

In [9]:
# 이 위까지 있었던 일련의 처리를 한번에 처리하도록 하는 함수
def preprocess(text):
    text = text.lower()
    text = text.replace('.', ' .')
    words = text.split(' ')

    word_to_id = {}
    id_to_word = {}

    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word

    corpus = [word_to_id[w] for w in words]
    corpus = np.array(corpus)

    return corpus, word_to_id, id_to_word



## 분포 가설
**`정의`** 단어의 의미는 주변 단어에 의해 형성된다.

## 동시 발생 행렬
각 단어 전후에 나온 단어들을 1로 표시하고 아닌 단어를 0으로 표현한 행렬

In [11]:
corpus, word_to_id, id_to_word = preprocess(text)

print(corpus)

print(id_to_word)

[0 1 2 3 4 1 5 6 7]
{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '', 7: '.'}


In [12]:
# text에 해당하는 문장의  동시발생행렬을 손으로 만들어 보자
C = np.array([
    [0,1,0,0,0,0,0],
    [1,0,1,0,1,1,0],
    [0,1,0,1,0,0,0],
    [0,0,1,0,1,0,0],
    [0,1,0,1,0,0,0],
    [0,1,0,0,0,0,1],
    [0,0,0,0,0,1,0],
], dtype=np.int32)

In [13]:
print(C[0])
print(C[word_to_id['goodbye']])

[0 1 0 0 0 0 0]
[0 1 0 1 0 0 0]


In [15]:
# 말뭉치로부터 동시발생 행렬을 만드는 함수 구현
def create_co_matrix(corpus, vocab_size, window_size=1):
    corpus_size = len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)

    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size + 1):
            left_idx = idx - i
            right_idx = idx + i

            if left_idx >=0:
                left_word_id = corpus[left_idx]
                co_matrix[word_id, left_word_id] += 1

            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id, right_word_id] += 1

    return co_matrix

In [16]:
# 코사인 유사도
def cos_similarity(x, y, eps=1e-8):
    nx = x / (np.sqrt(np.sum(x ** 2)) + eps) # eps는 0으로 나누는 오류가 발생하지 않게 하기 위함
    ny = y / (np.sqrt(np.sum(y ** 2)) + eps)
    return np.dot(nx, ny)

In [17]:
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)

c0 = C[word_to_id['you']] #"you"의 단어 벡터
c1 = C[word_to_id["i"]] # "i"의 단어 벡터
print(cos_similarity(c0,c1))

0.7071067691154799


# 유사 단어의 랭킹 표시

#### 다음과 같은 함수를 만들 예정 인수는 아래와 같음
most_similar(query, word_to_id, id_to_word, word_matrix, top=5)
- query: 검색어(단어)
- word_to_id: 단어에서 단어ID로의 딕셔너리
- id_to_word: 단어ID에서 단어로의 딕셔너리
- word_matrix: 단어벡터들을 한데 모은 행렬
- top: 상위 몇개까지 출력

In [18]:
def most_similar(query, word_to_id, id_to_word, word_matrix, top=5):
    # 검색어를 꺼낸다.
    if query not in word_to_id:
        print('%s(을)를 찾을 수 없습니다'% query)
        return

    print('\n[query]' + query)
    query_id = word_to_id[query]
    query_vec = word_matrix[query_id]

    # 코사인 유사도 계산
    vocab_size = len(id_to_word)
    similarity = np.zeros(vocab_size)
    for i in range(vocab_size):
        similarity[i] = cos_similarity(word_matrix[i], query_vec)

    # 코사인 유사도를 기준으로 내림차순 출력
    count = 0
    for i in (-1 * similarity).argsort():
        if id_to_word[i] == query:
            continue
        print(' %s: %s' % (id_to_word[i], similarity[i]))

        count += 1
        if count >= top:
            return


In [19]:
most_similar('you', word_to_id, id_to_word, C, top=5)


[query]you
 goodbye: 0.7071067691154799
 i: 0.7071067691154799
 hello: 0.7071067691154799
 say: 0.0
 and: 0.0


## 점별 상호 정보량(PMI)

In [20]:
def ppmi(C, verbose= False, eps=1e-8):
    M = np.zeros_like(C, dtype=np.float32)
    N = np.sum(C)
    S = np.sum(C, axis=0)
    total = C.shape[0] * C.shape[1]
    cnt = 0

    for i in range(C.shape[0]):
        for j in range(C.shape[1]):
            pmi = np.log2(C[i,j] * N / (S[j]*S[i]) + eps)
            M[i, j] = max(0, pmi)

            if verbose:
                cnt += 1
                if cnt % (total//100 + 1) == 0:
                    print('%.1f%% 완료' % (100*cnt/total))
    return M


In [22]:
W = ppmi(C)
W

array([[0., 2., 0., 0., 0., 0., 0., 0.],
       [2., 0., 1., 0., 1., 1., 0., 0.],
       [0., 1., 0., 2., 0., 0., 0., 0.],
       [0., 0., 2., 0., 2., 0., 0., 0.],
       [0., 1., 0., 2., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 2., 0.],
       [0., 0., 0., 0., 0., 2., 0., 3.],
       [0., 0., 0., 0., 0., 0., 3., 0.]], dtype=float32)

## 차원축소

##### 특잇값 분해